In [18]:
import requests
import googlemaps
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
import certifi


In [19]:
os.environ['SSL_CERT_FILE'] = certifi.where()

In [20]:
load_dotenv()

True

In [21]:
GOOGLE_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
def search_and_get_details(query: str, location: str = ""):
    gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
    # Build query
    search_query = f"{query} in {location}" if location else query
    results = gmaps.places(query=search_query)
    output = []
    for place in results.get('results', []):
        place_id = place['place_id']
        details = gmaps.place(place_id=place_id, fields=['name', 'formatted_phone_number', 'website'])
        result = details.get('result', {})
        if result:
            output.append({
                "name": result.get("name"),
                "phone": result.get("formatted_phone_number"),
                "website": result.get("website")
            })
    return output[:5]  # Limit for brevity

# Wrap as a LangChain tool
google_maps_tool = Tool(
    name="Google Maps Finder",
    func=lambda query: search_and_get_details(query=query),
    description="Use this tool to find professionals and their contact details. Input: search term and location, e.g., 'cardiologists in Frankfurt'."
)


In [23]:
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

agent = initialize_agent(
    tools=[google_maps_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [24]:
question = "find me all cardiologists in Frankfurt"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
I need to find cardiologists located in Frankfurt. 
Action: Google Maps Finder
Action Input: 'cardiologists in Frankfurt'
Observation: [{'name': 'CVC CardioVascular Center Frankfurt', 'phone': '069 46031340', 'website': 'https://www.cvcfrankfurt.de/'}, {'name': 'Kardiocentrum Frankfurt - Praxis Innenstadt', 'phone': '069 9130910', 'website': 'https://kardiocentrum.de/'}, {'name': 'Kardiologische Praxis', 'phone': '069 9043780', 'website': 'http://www.kardiopraxis.de/'}, {'name': 'Kardiocentrum Frankfurt an der Klinik Rotes Kreuz', 'phone': '069 944340', 'website': 'https://kardiocentrum.de/'}, {'name': 'Kardiologie am Main | Prof. Dr. med. Ulrich Fischer-Rasokat', 'phone': '069 66779973', 'website': 'https://kardiologie-am-main.de/'}]
Thought:I now have a list of cardiologists in Frankfurt along with their contact details and websites. 
Final Answer: Here are some cardiologists in Frankfurt:

1. **CVC CardioVascular Center Frankfurt**
   - Phone: